# Разведочный поиск на статьях ХабраХабра

In [5]:
import nltk
import urlparse
import collections
import BeautifulSoup

import nltk
import html2text
html2text.config.INLINE_LINKS = False
html2text.config.SKIP_INTERNAL_LINKS = True
html2text.config.IGNORE_ANCHORS = True
html2text.config.IGNORE_EMPHASIS = True
html2text.config.IGNORE_IMAGES = True
html2text.config.BYPASS_TABLES = False

import os
import json



### 1. Предобработка данных

In [6]:
DATASET_PATH = '../habr_pages/'
post_ids = sorted(int(filename) for filename in os.listdir(DATASET_PATH) if not filename.startswith('.'))
def load_post(post_id):
    with open(os.path.join(DATASET_PATH, str(post_id))) as fin:
        post = json.load(fin)
    return post

In [7]:
post_index = 1
post = load_post(1)

In [8]:
post

{u'author': u'deniskin',
 u'author_rating': 39.2,
 u'author_url': u'http://habrahabr.ru/users/deniskin/',
 u'comments': [{u'banned': False,
   u'link': u'#comment_3131',
   u'message_html': u'<div class="message html_format ">\n            \u041a\u0430\u043a \u0432\u0430\u0440\u0438\u0430\u043d\u0442, \u043c\u044b \u0434\u0443\u043c\u0430\u0435\u043c \u043e\u0441\u0442\u0430\u043d\u043e\u0432\u0438\u0442\u044c\u0441\u044f \u043d\u0430 WackoWiki<br>\r\n<a href="http://wackowiki.com/HomePage" title="http://wackowiki.com/HomePage" target="_blank">http://wackowiki.com/HomePage</a><br>\r\n<br>\r\n\u0415\u0441\u0442\u044c \u0437\u0430\u043c\u0435\u0447\u0430\u043d\u0438\u044f \u043f\u043e \u044d\u0442\u043e\u043c\u0443 \u0434\u0432\u0438\u0436\u043a\u0443?\n        </div>',
   u'replies': [{u'banned': False,
     u'link': u'#comment_3156',
     u'message_html': u'<div class="message html_format ">\n            \u0413\u043e\u0442\u043e\u0432 \u043f\u043e\u043c\u043e\u0433\u0430\u0442\u044c, \

In [9]:
content_text = html2text.html2text(post['content_html'])
tokens = nltk.tokenize.wordpunct_tokenize(content_text)

In [11]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [12]:
%%time

def html_to_plain(html):
    text = html2text.html2text(post['content_html'])

def post_to_corpus_line(post_id, post, morph):
    #post_id = post['id']
    author = post['author']
    tags = post['tags']

    # 1. words
    words = collections.Counter()
    
    soup = BeautifulSoup.BeautifulSoup(post['content_html'])
    text_parts = soup.findAll(text=True)

    content_text = ''.join(text_parts)
    space_chars = u'«»“”’*…/_.\\'
    for c in space_chars:
        content_text = content_text.replace(c, ' ')
    tokens = nltk.tokenize.wordpunct_tokenize(content_text)
    tokens = nltk.word_tokenize(content_text)
    for token in tokens:
        if len(token) > 2:
            token = token.lower().replace(u'ё', u'е')
            word = morph.parse(token)[0].normal_form
            if len(word) > 0:
                words[word] += 1
    
    # 2. users
    users = collections.Counter()
    
    def pass_comments(comments, users):
        for comment in comments:
            if not comment['banned'] and comment.get('username') is not None:
                username = comment['username']
                users[username] += 1
            pass_comments(comment['replies'], users)
    pass_comments(post['comments'], users)

    # 3. hubs
    
    def parse_hub_id(hub_pair):
        url_parts = filter(lambda s: len(s) > 0, urlparse.urlsplit(hub_pair[1]).path.split('/'))
        if len(url_parts) >= 2:
            hub_id = '_'.join(url_parts[-2:])
            return hub_id
        
    hubs = []
    for hub_pair in post['hubs']:
        hub_id = parse_hub_id(hub_pair)
        if hub_id:
            hubs.append(hub_id)
    
    # construct bag-of-words
    
    def construct_bow(words):
        return [
            (
                word.replace(' ', '_').replace(':', '_').replace('|', '_').replace('\t', '_') + 
                ('' if cnt == 1 else ':%g' % cnt)
            )
            for word, cnt in words.iteritems()
        ]
    
    parts = (
        ['\'%d' % post_id] + 
        ['|@words'] + construct_bow(words) +
        ['|@author'] + construct_bow({author: 1} if author is not None else {}) +
        ['|@users'] + construct_bow(users) + 
        ['|@tags'] + construct_bow({tag: 1 for tag in tags}) +
        ['|@hubs'] + construct_bow({hub_id: 1 for hub_id in hubs})
    )
    return ' '.join(parts)
    

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 10 µs


In [14]:
post_id = 1
print post_to_corpus_line(post_id, load_post(post_id), morph)

'1 |@words wiki вопрос:3 поскольку нет конкретный один задавать который составление кажется по-вашему часто сборник равно писать посему такой ответ обращаться привет доступный хабрахабра подходить faq какой wiki-движок коллективный решить смысл остаться:2 идеально как:2 человек:2 сайт:2 группа следующий использовать читатель весь идея что для:3 мы |@author deniskin |@users BeLove:2 Vilgelm ubunterro bohdan4ik I3Lack_CaT STFBEE Umed arren shus rasa Oriand minaton TechnologyMasta kukutz khekkly gameboyhippo:2 lukaville:3 Zelebober crwin geraxe:3 deniskin:4 vyacheslav_ka unStaiL Kuuuzya:2 Zoberg dasty danpetruk terrance cdkrot BrownTrigger |@tags wiki механизм движок FAQ ЧАВО хабрахаб |@hubs hub_habr


In [ ]:
for i, post_id in enumerate(post_ids):
    line = post_to_corpus_line(post_id, load_post(post_id), morph)
    with open('habr_text/' + str(post_id), 'w') as fout:
        fout.write(line.encode('utf8'))
        
with open('habrahabr_corpus_multimodal.txt', 'w') as fout:
    for post_id in post_ids:
        line = post_to_corpus_line(post_id, load_post(post_id), morph)
        fout.write(line.encode('utf8') + '\n')

### 2. Строим тематическую модель

Прежде всего необходимо подготовить входные данные. BigARTM имеет собственный формат документов для обработки, называемый батчами. В библиотеке присутствуют средства по созданию батчей из файлов в форматах Bag-Of-Words UCI и Vowpal Wabbit (подробности можно найти в http://docs.bigartm.org/en/latest/formats.html).

In [ ]:
import os
import artm
import glob


os.environ['ARTM_SHARED_LIBRARY'] = "/Users/yanina-n/study/diplom/bigartm/build/src/artm/libartm.dylib"


BATCHES_FOLDER = '../habrahabr-dataset-master/pydata_batches'
HABR_DATA_PATH = '../habrahabr-dataset-master/habrahabr_corpus_multimodal.txt'
DICT_PATH = BATCHES_FOLDER + '/dictionary.dict'

В Python API, по аналогии с алгоритмами из scikit-learn, входные данные представлены одним классом BatchVectorizer. Объект этого класса принимает на вход батчи или файлы UCI / VW и подаётся на вход всем методам. В случае, если входные данные не являются батчами, он создаёт их и сохраняет на диск для последующего быстрого использования.


In [ ]:
batch_vectorizer = artm.BatchVectorizer(data_path=HABR_DATA_PATH, data_format='vowpal_wabbit', collection_name='habr', target_folder=BATCHES_FOLDER)

In [ ]:
batch_vectorizer = None
if len(glob.glob(os.path.join(BATCHES_FOLDER, '*.batch'))) < 1:
    batch_vectorizer = artm.BatchVectorizer(data_path=HABR_DATA_PATH, data_format='vowpal_wabbit', collection_name='habr', target_folder=BATCHES_FOLDER)
else:
    batch_vectorizer = artm.BatchVectorizer(data_path=HABR_DATA_PATH, data_format='vowpal_wabbit')

In [ ]:
dictionary = artm.Dictionary()

model_plsa = artm.ARTM(topic_names=['topic_{}'.format(i) for i in xrange(200)],
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    use_unigram_document_model=False,
                                                    dictionary=dictionary)],
                       cache_theta=True)

model_artm = artm.ARTM(topic_names=['topic_{}'.format(i) for i in xrange(200)],
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    use_unigram_document_model=False,
                                                    dictionary=dictionary)],
                       regularizers=[artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=-0.15),
                                     artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=-0.1)],
                       cache_theta=True)

In [ ]:
model_artm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=1.5e+5))

In [ ]:
model_artm.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
model_artm.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model_artm.scores.add(artm.TopicKernelScore(name='TopicKernelScore', probability_mass_threshold=0.3))

Следующий шаг — инициализация моделей. Сделаем это по словарю, что означает, что
- будет создана матрица $\Phi$ с именем 'pwt', число строк и столбцов в ней будет взято исходя из числа слов в словаре и заданного в модели числа тем;
- эта матрица будет заполнена случайными значениями из диапазона (0, 1) и нормализована.

Надо отметить, что этот шаг является опциональным, поскольку модель может быть автоматически инициализирована во время вызовов fit_offline() / fit_online().

Словарь – это объект BigARTM, содержащий информацию о коллекции (словарь коллекции, различные величины и счётчики, связанные со словами). Создать словарь можно на основе папки с батчами. Затем собранный словарь можно сохранять на диск и позже подгрузить вновь.

In [ ]:
if not os.path.isfile(DICT_PATH):
    dictionary.gather(data_path=batch_vectorizer.data_path)
    dictionary.save(dictionary_path=DICT_PATH)

dictionary.load(dictionary_path=DICT_PATH)

In [ ]:
model_plsa.initialize(dictionary=dictionary)
model_artm.initialize(dictionary=dictionary)

Теперь попробуем обучить модели в оффлайн-режиме (т.е. обновляя Фи раз за проход по коллекции). Инициируем пятнадцать проходов:


In [ ]:
model_plsa.num_document_passes = 1
model_artm.num_document_passes = 1

model_plsa.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)
model_artm.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)

In [ ]:
plt.plot(xrange(model_plsa.num_phi_updates), model_plsa.score_tracker['SparsityPhiScore'].value, 'b--',
                 xrange(model_artm.num_phi_updates), model_artm.score_tracker['SparsityPhiScore'].value, 'r--', linewidth=2)
plt.xlabel('Iterations count')
plt.ylabel('PLSA Phi sp. (blue), ARTM Phi sp. (red)')
plt.grid(True)
plt.show()

plt.plot(xrange(model_plsa.num_phi_updates), model_plsa.score_tracker['SparsityThetaScore'].value, 'b--',
                 xrange(model_artm.num_phi_updates), model_artm.score_tracker['SparsityThetaScore'].value, 'r--', linewidth=2)
plt.xlabel('Iterations count')
plt.ylabel('PLSA Theta sp. (blue), ARTM Theta sp. (red)')
plt.grid(True)
plt.show()

<img src="pics/comp.png">

In [ ]:
def print_measures(model_plsa, model_artm):
    print 'Sparsity Phi: {0:.3f} (PLSA) vs. {1:.3f} (ARTM)'.format(
        model_plsa.score_tracker['SparsityPhiScore'].last_value,
        model_artm.score_tracker['SparsityPhiScore'].last_value)

    print 'Sparsity Theta: {0:.3f} (PLSA) vs. {1:.3f} (ARTM)'.format(
        model_plsa.score_tracker['SparsityThetaScore'].last_value,
        model_artm.score_tracker['SparsityThetaScore'].last_value)

    print 'Kernel contrast: {0:.3f} (PLSA) vs. {1:.3f} (ARTM)'.format(
        model_plsa.score_tracker['TopicKernelScore'].last_average_contrast,
        model_artm.score_tracker['TopicKernelScore'].last_average_contrast)

    print 'Kernel purity: {0:.3f} (PLSA) vs. {1:.3f} (ARTM)'.format(
        model_plsa.score_tracker['TopicKernelScore'].last_average_purity,
        model_artm.score_tracker['TopicKernelScore'].last_average_purity)

    print 'Perplexity: {0:.3f} (PLSA) vs. {1:.3f} (ARTM)'.format(
        model_plsa.score_tracker['PerplexityScore'].last_value,
        model_artm.score_tracker['PerplexityScore'].last_value)

    plt.plot(xrange(model_plsa.num_phi_updates), model_plsa.score_tracker['PerplexityScore'].value, 'b--',
             xrange(model_artm.num_phi_updates), model_artm.score_tracker['PerplexityScore'].value, 'r--', linewidth=2)
    plt.xlabel('Iterations count')
    plt.ylabel('PLSA perp. (blue), ARTM perp. (red)')
    plt.grid(True)
    plt.show()
    
print_measures(model_plsa, model_artm)

### 3. Разведочный поиск

In [ ]:
import copy

thetas = pandas.read_csv('../habrahabr-dataset-master/theta_matrix.csv', sep=' ').set_index('id')
articles = copy.deepcopy(thetas)
thetas = thetas[[col for col in thetas.columns if col.startswith('objective')]]

relevant_pages = [
    'geektimes/14',
    'habrahabr/65',
]

thetas.ix[relevant_pages, :]

In [15]:
import numpy as np

def what_to_watch_next(relevant_pages, irrelevant_pages, observed_confidence=10000, top_size=30, penalty=1):
    y = -np.ones(thetas.shape[0])
    c = np.ones(thetas.shape[0])

    for post_id in relevant_pages:
        sel = thetas.index == post_id
        y[sel] = +1
        c[sel] = observed_confidence
        
    for post_id in irrelevant_pages:
        sel = thetas.index == post_id
        y[sel] = -1
        c[sel] = observed_confidence

    y_pred = Ridge(alpha=penalty).fit(thetas.values, y, sample_weight=c).predict(thetas.values)
    order = np.argsort(-y_pred)
    
    return list(thetas.index[order[:top_size]])

/Users/yanina-n/study/diplom/shad_backup_romovpa


##### Просмотр заголовков статей

In [ ]:
import pymongo

client = pymongo.MongoClient()
tmblogs = client['tmblogs']
posts = tmblogs['posts']

In [ ]:
def show_posts(post_indices):
    '''from IPython.display import HTML
    html = '<ol>'
    for post_id in post_indices:
        post = tmblogs['posts'].find_one(post_id)
        html += u'<li><tt>{_id}</tt> <a href="{url}">{title}</a></li>'.format(**post)
    html += '</ol>'
    return HTML(''.join(html))   ''' 
    
    for post in post_indices:
        print post, articles.ix[post]['title']

##### Пример итераций поиска

Нашли вручную одну статью

In [ ]:
relevant_pages = [
    'habrahabr/58',
]

Составили список похожих статей

In [ ]:
next_pages = what_to_watch_next(relevant_pages, [], observed_confidence=10000)

In [ ]:
show_posts(next_pages)

<img src="pics/titles1.png">

Помечаем найденные статьи как релевантные/нерелевантные:

In [ ]:
irrelevant_pages = [
    'geektimes/7274',
    'megamozg/4956',
    'megamozg/2026',
    'geektimes/6643',
    'megamozg/1140',
    'geektimes/6082',
    'geektimes/1843',
    
    
]

relevant_pages += [
    'megamozg/4840', 
    'geektimes/6118',
    'geektimes/3983',
    'geektimes/6643',
    'megamozg/4840'
]

Обновляем поисковую выдачу

In [ ]:
%%time
next_pages = what_to_watch_next(relevant_pages, irrelevant_pages, observed_confidence=10000, penalty=1)

In [ ]:
show_posts(next_pages)

<img src="pics/titles2.png">

И так несколько итераций...